In [1]:
################# Imports #############
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F


REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

#Data preprocessing
class DogsVSCats():
    IMG_SIZE = 50
    CATS = "/home/monsur/PetImages/Cat"#Paths 
    DOGS = "/home/monsur/PetImages/Dog"
    #TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}#defining cats as 0 and Dogs as 1
    training_data = [] #creating a list to contain the training data.

    catcount = 0 # total data counting for data regularization.
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data) #saving training data as npy file.
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()
                    

In [2]:
training_data = np.load("training_data.npy", allow_pickle = True) #loading the preprocessed data, by this we only once have to preprocess data

In [3]:

##############Model Declaration #######################

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5) #1st layer
        self.conv2 = nn.Conv2d(32,64,5)#2nd layer
        self.conv3 = nn.Conv2d(64,128,5)#3rd layer
        
        self.fc1 = nn.Linear(2*2*128,256) #Fully connected layer1
        self.fc2 = nn.Linear(256,256)
        self.fc3 = nn.Linear(256,2)#Fully connected layer2
    
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))#Maxpooling for first layer
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))#Maxpooling for second layer
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))#Maxpooling for third layer
        x= x.view(-1,2*2*128)
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x= self.fc3(x)
        
        #if self._to_linear is None:
        #    self._to_linear = x[0].shape[0]
        
        return F.log_softmax(x,dim=1)#Softmaxing the model to get one hot vector.
        
        
net = Net()
net=net.cuda()
print(net)

#Parameter count function
def count_parameters(net):
    for name, param in net.named_parameters():
        if param.requires_grad:
            print(name, param.numel())
    return sum(p.numel() for p in net.parameters() if p.requires_grad)


print("Total Parameters: ",count_parameters(net))

        

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)
conv1.weight 800
conv1.bias 32
conv2.weight 51200
conv2.bias 64
conv3.weight 204800
conv3.bias 128
fc1.weight 131072
fc1.bias 256
fc2.weight 65536
fc2.bias 256
fc3.weight 512
fc3.bias 2
Total Parameters:  454658


In [4]:
import torch.optim as optim #optimizer starts after model initialization for loss counting

optimizer = optim.Adam(net.parameters(), lr =0.001) #lr is learning rate typically lr = 1e^-3
loss_function = nn.MSELoss() #Using Mean squared error loss 

X= torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X=X/255.0
Y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
VAL_SIZE = int(len(X)*VAL_PCT)
print(VAL_SIZE)

2494


In [5]:
train_X = X[:-VAL_SIZE]
train_Y = Y[:-VAL_SIZE]

test_X = X[-VAL_SIZE:]
test_Y = Y[-VAL_SIZE:]

train_X = train_X.cuda()
train_Y = train_Y.cuda()
test_X = test_X.cuda()
test_Y = test_Y.cuda()

print(len(train_X))
print(len(test_X))

22452
2494


In [6]:
BATCH_SIZE = 64
EPOCHS = 5

for epochs in range(EPOCHS):
    for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
        #print(i,i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_Y = train_Y[i:i+BATCH_SIZE]
        
        batch_X=batch_X.cuda()
        batch_Y=batch_Y.cuda()
        
        optimizer.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs,batch_Y)
        loss.backward()
        optimizer.step()
    print(loss)
        

        
    

  5%|▌         | 18/351 [00:00<00:01, 172.93it/s]

tensor(1.6758, device='cuda:0', grad_fn=<MseLossBackward>)


  5%|▌         | 18/351 [00:00<00:01, 172.35it/s]

tensor(1.6663, device='cuda:0', grad_fn=<MseLossBackward>)


  5%|▍         | 17/351 [00:00<00:02, 159.56it/s]

tensor(1.6438, device='cuda:0', grad_fn=<MseLossBackward>)


  5%|▌         | 18/351 [00:00<00:01, 166.98it/s]

tensor(1.6437, device='cuda:0', grad_fn=<MseLossBackward>)


100%|██████████| 351/351 [00:02<00:00, 127.26it/s]


tensor(1.6441, device='cuda:0', grad_fn=<MseLossBackward>)


In [7]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_Y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]
        #net_out = net_out.cuda()
        #real_class= real_class.cuda()
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct+=1
        
        total+=1
        
print("Accuracy: ", round(correct/total,3))

100%|██████████| 2494/2494 [00:01<00:00, 1432.61it/s]

Accuracy:  0.76
